In [1]:
from keras.preprocessing.image import ImageDataGenerator

img_w, img_h = 150, 150        # 画像をリサイズするときのサイズ
batch_size = 32                # ミニバッチのサイズ

# 訓練データを読み込んで加工するジェネレーターを生成
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,         # RGB値を0～1の範囲に変換
    rotation_range=15,         # 15度の範囲でランダムに回転させる
    zoom_range=0.2,            # ランダムに拡大
    horizontal_flip=True       # 水平方向にランダムに反転
    )

# テストデータを読み込むジェネレーターを生成
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# 訓練データをミニバッチの数だけ生成
train_generator = train_datagen.flow_from_directory(
    'data/train',               # 訓練データのフォルダー
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

# テストデータをミニバッチの数だけ生成
validation_generator = test_datagen.flow_from_directory(
    'data/validation',          # テストデータのフォルダー
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

Using TensorFlow backend.


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [2]:
# 正解ラベルを確認
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


In [3]:
## CNNの構築

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras import optimizers

# モデルを構築
model = Sequential()

# （第1層）畳み込み層
model.add(
    Conv2D(
        filters=32,                # フィルターの数は32
        kernel_size=(3, 3),        # 3×3のフィルターを使用
        input_shape=(150, 150, 3), # 入力データの形状
        padding='same',            # ゼロパディングを行う
        activation='relu'          # 活性化関数はReLU
        ))
# （第2層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第3層）畳み込み層
model.add(
    Conv2D(
        filters=32,            # フィルターの数は32
        kernel_size=(3, 3),    # 3×3のフィルターを使用
        activation='relu'      # 活性化関数はReLU
        ))
# （第4層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第5層）畳み込み層
model.add(
    Conv2D(filters=64,         # フィルターの数は64
           kernel_size=(3, 3), # 3×3のフィルターを使用
           activation='relu'   # 活性化関数はReLU
          ))
# （第6層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト25％
model.add(Dropout(0.25))

# Flatten層
# 出力層への入力を4階テンソルから2階テンソルに変換する
model.add(Flatten())

# （第7層）全結合層
model.add(
    Dense(64,                  # ニューロン数は64
          activation='relu'))  # 活性化関数はReLU
# ドロップアウト50％
model.add(Dropout(0.5))

# （第8層）出力層
model.add(
    Dense(
        1,                     # ニューロン数は1個
        activation='sigmoid')) # 活性化関数はsigmoid

# モデルのコンパイル
model.compile(
    loss='binary_crossentropy', # バイナリ用の交差エントロピー誤差
    metrics=['accuracy'],       # 学習評価として正解率を指定
    # Adamアルゴリズムで最適化
    optimizer=optimizers.Adam(),
)

# モデルのサマリを表示
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 73, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [4]:
## 学習の実行

epochs = 60             # 学習回数
batch_size = batch_size # 設定済みのミニバッチのサイズ
validation_num = 2000   # 訓練データの総数

# 学習を行う
history = model.fit_generator(
    train_generator,    # 訓練データ
    epochs=epochs,      # 学習回数
    # ジェネレータのサイズを設定
    steps_per_epoch = len(train_generator),
    # 訓練データの総数
    validation_steps = validation_num,
    # 学習の進捗状況を出力する    
    verbose=1,
    # テストデータ
    validation_data=validation_generator,
)

Epoch 1/60
63/63 [==============================] - 742s 12s/step - loss: 0.7639 - acc: 0.5005 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/60
63/63 [==============================] - 746s 12s/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6922 - val_acc: 0.6150
Epoch 3/60
63/63 [==============================] - 792s 13s/step - loss: 0.6914 - acc: 0.5233 - val_loss: 0.6903 - val_acc: 0.5387
Epoch 4/60
63/63 [==============================] - 738s 12s/step - loss: 0.6919 - acc: 0.5258 - val_loss: 0.6876 - val_acc: 0.5600
Epoch 5/60
63/63 [==============================] - 733s 12s/step - loss: 0.6868 - acc: 0.5412 - val_loss: 0.6822 - val_acc: 0.6062
Epoch 6/60
63/63 [==============================] - 731s 12s/step - loss: 0.6809 - acc: 0.5685 - val_loss: 0.6776 - val_acc: 0.6188
Epoch 7/60
63/63 [==============================] - 730s 12s/step - loss: 0.6793 - acc: 0.5620 - val_loss: 0.6627 - val_acc: 0.6012
Epoch 8/60
63/63 [==============================] - 764s 12s/step - loss: 0.